This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
from config.config import api_key
API_KEY = api_key

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
res = requests.get(f'https://www.quandl.com/api/v3/datasets/FSE/AFX_X?api_key={api_key}')

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
# print(res.json())

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## Load the Data

In [5]:
res_fse = requests.get(f'https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key={api_key}')

In [6]:
fse_dict = res_fse.json()

## Data Exploration

In [7]:
print(f"The keys in the dataset are:\n{list(fse_dict['dataset'].keys())}")

The keys in the dataset are:
['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id']


In [8]:
print(f"The columns for the data are:\n{fse_dict['dataset']['column_names']}")

The columns for the data are:
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [9]:
print(f"There are {len(fse_dict['dataset']['data'])} rows in the dataset.")

There are 255 rows in the dataset.


## Analysis
### Highest and Lowest Prices

In [10]:
def find_yearly_high_low(data_dict):
    """
    Find the maximum and minimum prices for the year.
    
    Parameters
    ----------
    data_dict: dict
        The output from the API call.
    
    """
    
    # Find the indices for the high and low daily prices.
    high_index = data_dict['dataset']['column_names'].index('High')
    low_index = data_dict['dataset']['column_names'].index('Low')
    
    # Initialize the highest and lowest prices with the first values for High and Low in the dataset.
    highest = data_dict['dataset']['data'][0][high_index]
    lowest = data_dict['dataset']['data'][0][low_index]
    for row in data_dict['dataset']['data']:
        highest = max(highest, row[high_index])
        lowest = min(lowest, row[low_index])
    year = data_dict['dataset']['start_date'].split('-')[0]
    print(f'The highest price for {year} was {highest}.')
    print(f'The lowest price for {year} was {lowest}.')

In [11]:
find_yearly_high_low(fse_dict)

The highest price for 2017 was 53.54.
The lowest price for 2017 was 33.62.


### Largest Daily Change

In [12]:
def find_largest_change(data_dict):
    """
    Find the largest daily change for the year.
    
    Parameters
    ----------
    data_dict: dict
        The output from the API call.
    
    """
    
    # Find the indices for the high and low daily prices.
    high_index = data_dict['dataset']['column_names'].index('High')
    low_index = data_dict['dataset']['column_names'].index('Low')
    
    delta = data_dict['dataset']['data'][0][high_index] - data_dict['dataset']['data'][0][low_index]
    for row in data_dict['dataset']['data']:
        row_delta = row[high_index] - row[low_index]
        delta = max(delta, row_delta)
    year = data_dict['dataset']['start_date'].split('-')[0]
    print(f'The largest daily change for {year} was {delta}.')

In [13]:
find_largest_change(fse_dict)

The largest daily change for 2017 was 2.8100000000000023.


### Largest Change Between Two Days

In [14]:
def find_largest_closing_change(data_dict):
    """
    Find the largest change in closing prices between adjacent days.
    
    Parameters
    ----------
    data_dict: dict
        The output from the API call.
    
    """    
    
    # Find index values for Close.
    close_index = data_dict['dataset']['column_names'].index('Close')
    
    closing_prices = [x[close_index] for x in data_dict['dataset']['data']]
    current_day_prices = closing_prices[:-1]
    next_day_prices = closing_prices[1:]
    price_pairs = list(zip(current_day_prices, next_day_prices))
    delta_prices = [x[1] - x[0] for x in price_pairs]
    max_delta = max(delta_prices)
    
    year = data_dict['dataset']['start_date'].split('-')[0]
    print(f'The largest change between closing prices on adjacent days for the year {year} was {max_delta}.')

In [15]:
find_largest_closing_change(fse_dict)

The largest change between closing prices on adjacent days for the year 2017 was 2.559999999999995.


### Find Average Trading Volume

In [16]:
def average_trading_volume(data_dict):
    """
    Find the average traded volume for the year.
    
    Parameters
    ----------
    data_dict: dict
        The output from the API call.
    
    """ 
    
    
    traded_volume_index = data_dict['dataset']['column_names'].index('Traded Volume')
    traded_volumes = [x[traded_volume_index] for x in data_dict['dataset']['data']]
    average_traded_volume = sum(traded_volumes) / len(traded_volumes)
    
    year = data_dict['dataset']['start_date'].split('-')[0]
    print(f'The average traded volume for the year {year} was {average_traded_volume}.')

In [17]:
average_trading_volume(fse_dict)

The average traded volume for the year 2017 was 89124.33725490196.


### Find Median Trading Volume

In [18]:
def compute_median(numbers):
    numbers_sorted = sorted(numbers)
    
    if len(numbers_sorted) % 2 == 0:
        lower_index = int(len(numbers_sorted) / 2)
        median = sum(numbers_sorted[lower_index:lower_index + 2]) / 2
    else:
        median_index = int(len(numbers_sorted) / 2)
        median = numbers_sorted[median_index]
    return median

In [19]:
def median_trading_volume(data_dict):
    """
    Find the median traded volume for the year.
    
    Parameters
    ----------
    data_dict: dict
        The output from the API call.
    
    """ 
    
    
    traded_volume_index = data_dict['dataset']['column_names'].index('Traded Volume')
    traded_volumes = [x[traded_volume_index] for x in data_dict['dataset']['data']]
    median_traded_volume = compute_median(traded_volumes)
    
    year = data_dict['dataset']['start_date'].split('-')[0]
    print(f'The median traded volume for the year {year} was {median_traded_volume}.')

In [20]:
median_trading_volume(fse_dict)

The median traded volume for the year 2017 was 76286.0.
